# Goal
The goal of this project is to identify unusual patterns in catalog or potential data manipulation inacuracies

## Project Checklist
- Frame the project to get the big picture
  - Why identify anomalies in the data? 
    - to validate the reports made from the data and define biases based on the (in)completness of the data 
    - to identify patterns missed in the reports.
- Get the [data](https://docs.google.com/spreadsheets/d/173kXrmgG0K4Q_K0d2GFgADnW3wB66xh9R1IdyjB3eXY/edit?gid=1#gid=1)
- Explore the data
- Prepare the data to better expose the underlying patterns
- Explore models and pick the best one(s)
- Present solution

In [ ]:
# Install necessary packages
%pip install ipywidgets widgetsnbextension jupyter_contrib_nbextensions
%pip install pandas numpy matplotlib seaborn scikit-learn streamlit
# Enable Jupyter Notebook extensions
!jupyter contrib nbextension install --user
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn

import requests
import re

%pip install tqdm
from tqdm.notebook import tqdm

import time

import warnings
warnings.filterwarnings('ignore')

%pip freeze > requirements.txt

---
### Get the data


In [ ]:
def is_valid_url(url: str):
    """
    Checks if the URL is valid.
    
    Parameters:
        url (str): The URL to check.
    
    Returns:
        bool: True if the URL is valid, False otherwise.
    """
    return bool(re.match(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', url))

In [ ]:
def convert_spreadsheet_to_csv(spreadsheet_url: str):
    """ 
        This function will convert the google spreadsheet url to the csv url
        Parameters:
            spreadsheet_url (string): The google spreadsheet url
        Returns:
            string: The csv url
    """
    return re.sub(r'/edit.*$', '/export?format=csv', spreadsheet_url)

def get_the_data(sheet_url:str='https://docs.google.com/spreadsheets/d/173kXrmgG0K4Q_K0d2GFgADnW3wB66xh9R1IdyjB3eXY/edit?gid=1#gid=1'):
    """ 
        This function will get the data from the google sheet and return it as a pandas dataframe 
        Parameters:
            sheet_url (string): The url of the google sheet
        Returns:
            pandas dataframe: The data from the google sheet
    """
    
    csv_export_url = convert_spreadsheet_to_csv(sheet_url) # Convert the google sheet url to csv url
    
    if sheet_url == '':
        return pd.DataFrame()
    try:       
        if not is_valid_url(sheet_url):
            print(f"Invalid URL: {sheet_url}")
            return pd.DataFrame()
        
        r = requests.head(csv_export_url, verify=False, allow_redirects=True)
        
        if r.status_code != 200:
            return pd.DataFrame()
        
    except requests.exceptions.ConnectionError:
        return pd.DataFrame()
    
    try:
        response = requests.get(csv_export_url, verify=False) # Make the request to the url
        if response.status_code == 200:
            df= pd.read_csv(csv_export_url)
        elif response.status_code == 401:
            print(f'Authentication issue {csv_export_url} : {response.status_code}') # - {response.text}')
            df = pd.DataFrame()
        else:
            print(f'Failed to retrieve data from {csv_export_url} : {response.status_code}') # - {response.text}')
            df = pd.DataFrame()
    except Exception as e:
        print(f'Failed to retrieve data from {csv_export_url} : {e}')
        df = pd.DataFrame()
    
    return df

dataset_base = get_the_data()
dataset_base.head(2)

In [ ]:
def check_schema(url: str):
    """
        This function will check the schema of the url and returns a correct scheme 
    """
    if not url.startswith(('http://', 'https://')):
        url = 'https://' + url

    # print(f"URL: {url}")
    return url

def convert_bitly_to_url(bitly_url: str):
    """ 
        This function will convert the bitly url to the original google sheet url
        Parameters:
            bitly_url (string): The bitly url
        Returns:
            string: The original google sheet url
    """
    try:
        res:str = requests.get(bitly_url, verify=False, allow_redirects=True).url
    except requests.exceptions.SSLError as ssl_error: # The URL is not valid
        # print(f"SSL Error: {ssl_error}")
        res = bitly_url
    return res

def convert_geni_to_url(geni_url:str):
    """ 
        This function will convert the geni url to the original google sheet url
        Parameters:
            geni_url (string): The geni url
        Returns:
            string: The original google sheet url
    """
    try:
        res:str = requests.get(geni_url, verify=False, allow_redirects=True).url
    except requests.exceptions.SSLError as ssl_error: # The URL is not valid
        print(f"SSL Error: {ssl_error}")
        res = geni_url
    return res
   
def convert_airtable_to_url(airtable_url: str):
    """ 
        This function will convert the airtable url to the original google sheet url
        Parameters:
            airtable_url (string): The airtable url
        Returns:
            string: The original google sheet url
    """
    res:str = re.sub(r'/tbl([^/]+)/([^/]+)', r'/tbl\1/\2/csv', airtable_url)
    print(f"Airtable URL: {airtable_url} - \nConverted URL: {res}")
    return res
        
def get_datasets(base_df: pd.DataFrame=dataset_base, target_column: str = 'bitly '):
    """
    This function will get the datasets from the 'bitly' column in the dataframe and return the datasets.
    
    Parameters:
        base_df (pandas dataframe): The dataframe containing the data.
        target_column (string): The name of the target column.
    
    Returns:
        list of dict: A list of dictionaries with 'entry' and 'title' as keys.
    """
    datasets = []
    for _, row in base_df.iterrows():
        entry = row[target_column].strip()
        title = row['title']
        entry = check_schema(entry)

        conversion_functions = {
            'spreadsheets': convert_spreadsheet_to_csv,
            'bit.ly': convert_bitly_to_url,
            'BIT.LY': convert_bitly_to_url,
            'airtable.com': convert_airtable_to_url,
            'geni': convert_geni_to_url,
        }

        for keyword, func in conversion_functions.items():
            if keyword in entry:
                entry = func(entry)
                break

        if 'informationisbeautiful' in entry:
            # skip the entry
            entry = ''

        # print(f"Getting the dataset from URL: {entry}")
        dataset = get_the_data(entry)

        if not dataset.empty:
            datasets.append({'entry': dataset, 'title': title})

    return datasets

# print(f'Base Dataset Shape: {dataset_base.shape}') -> (151, 6)
multiple_datasets = get_datasets()

---
### Explore the data

#### Preprocessing

In [ ]:
for entry, dataset in enumerate(multiple_datasets):
    print(f"{entry:3d} Title: {dataset['title']} Shape: {dataset['entry'].shape}")

#### Prepare the data to better expose the underlying patterns

In [ ]:
def remove_duplicates(datasets:list):
    """
        This function will remove the datasets with the same shapes, leaving a single copy.
        Parameters:
            datasets (list): The list of datasets
        Returns:
            list: The list of datasets without duplicates 
    """
    unique_datasets = []
    for dataset in datasets:
        if dataset.shape not in [data.shape for data in unique_datasets]:
            unique_datasets.append(dataset)
    return unique_datasets

def remove_small_datasets(datasets:list):
    """
        This function will remove the datasets with less than 10 rows.
        Parameters:
            datasets (list): The list of datasets
        Returns:
            list: The list of datasets without small datasets 
    """
    return [dataset for dataset in datasets if dataset.shape[0] > 10 & dataset.shape[1] > 2]

def handle_missing_values(datasets:list):
    """
        This function will handle the missing values in the datasets.
        Parameters:
            datasets (list): The list of datasets
        Returns:
            list: The list of datasets without missing values 
    """
    for dataset in datasets:
        dataset.fillna(method='ffill', inplace=True) 
    return datasets

def find_non_alphanumeric_data(datasets: list):
    """
    This function will list the datasets with columns containing non-alphanumeric data.
    
    Parameters:
        datasets (list): The list of datasets
    
    Returns:
        list: The list of non-alphanumeric characters found in the datasets
    """
    non_alphanumeric_chars = set()
    for idx, dataset in enumerate(datasets):
        for column in dataset.columns:
            for index, value in dataset[column].items():
                if isinstance(value, str):  # Check if the value is a string
                    non_alphanumeric = re.findall(r'[^a-zA-Z0-9]', value)
                    if non_alphanumeric:
                        non_alphanumeric_chars.update(non_alphanumeric)
                        # print(f"Dataset {idx}, Row {index}, Column '{column}': \
                        #       Non-alphanumeric characters: {non_alphanumeric}")
    return list(non_alphanumeric_chars)

def replace_non_alphanumeric_data(datasets: list, non_alphanumeric_chars: list):
    """
    This function will replace non-alphanumeric characters in the datasets.
    
    Parameters:
        datasets (list): The list of datasets
        non_alphanumeric_chars (list): The list of non-alphanumeric characters to replace
    
    Returns:
        datasets (list): The list of datasets with non-alphanumeric characters replaced
    """
    for dataset in datasets:
        for column in dataset.columns:
            for index, value in dataset[column].items():
                if isinstance(value, str):  # Check if the value is a string
                    for char in non_alphanumeric_chars:
                        value = value.replace(char, '')
                    dataset.at[index, column] = value
    return datasets

datasets = [dataset['entry'] for dataset in multiple_datasets]

print(f"Number of datasets: {len(datasets)}", end='\t')
datasets = remove_duplicates(datasets)
datasets = remove_small_datasets(datasets)
datasets = handle_missing_values(datasets)

non_alphanumeric_chars:list = find_non_alphanumeric_data(datasets)
datasets = replace_non_alphanumeric_data(datasets, non_alphanumeric_chars)

print(f'Editted number of datasets: {len(datasets)}')

In [ ]:
def get_column_names(datasets:list=datasets):
    """
        This function will get the column names of each dataset in the list of datasets.
        Parameters:
            datasets (list): The list of datasets
        Returns:
            List of list: The list of column names of each dataset
    """
    for dataset in datasets:
        categorical_columns, numeric_columns = [],[]

        numeric_columns.append(dataset.select_dtypes(include=[np.number]).columns)
        categorical_columns.append(dataset.select_dtypes(include=[object]).columns)

        print(f"Numeric Columns: {list(numeric_columns)}\nCategorical Columns: {list(categorical_columns)}\n\n")
    return [list(dataset.columns) for dataset in datasets]

def clean_columns(datasets:list=datasets):
    """
        This function will remove the columns with links
        Parameters:
            datasets (list): The list of datasets
        Returns:
            datasets (list): The list of datasets without columns with links
    """
    for dataset in datasets:
        for column in dataset.columns:
            if 'http' in column:
                dataset.drop(column, axis=1, inplace=True)
    return datasets

columns = get_column_names(datasets)
columns = clean_columns(datasets)

In [ ]:
def plot_data_distribution(datasets: list):
    """
    This function will plot the data distribution of the datasets.
    For each dataset, plot a bunch of subplots for each column.
    
    Parameters:
        datasets (list): The list of datasets.
    """
    for idx, dataset in enumerate(datasets):
        if idx < 10:
            numeric_columns = dataset.select_dtypes(include=['number']).columns
            num_columns = len(numeric_columns)

            if num_columns == 0:
                print(f"Dataset {idx} has no numeric columns to plot.")
                continue
            
            num_rows = (num_columns // 4) + (num_columns % 4 > 0)
            fig, axs = plt.subplots(num_rows, min(4, num_columns), figsize=(15, 5 * num_rows))
            axs = axs.flatten() if num_columns > 1 else [axs]

            for col_idx, col_name in enumerate(numeric_columns):
                axs[col_idx].plot(dataset[col_name])
                axs[col_idx].set_title(f'Dataset-{idx} {dataset.shape}', fontsize=8)
                axs[col_idx].set_xlabel(f'{col_name.replace('$', '\\$')}', fontsize=8)
                axs[col_idx].set_ylabel('Value')
        
            plt.tight_layout()
            plt.show() 

plot_data_distribution(datasets)

In [ ]:
# Convert the categorical columns to numerical columns
from sklearn.preprocessing import LabelEncoder # Encode target labels with value between 0 and n_classes-1.
from sklearn.preprocessing import StandardScaler # Removing the mean and scaling to unit variance

def convert_categorical_columns(datasets:list=datasets):
    """
        This function will convert the categorical columns to numerical columns.
        Parameters:
            datasets (list): The list of datasets
        Returns:
            datasets (list): The list of datasets with numerical columns
    """
    for dataset in datasets:
        categorical_columns = dataset.select_dtypes(include=[object]).columns
        for column in categorical_columns:
            encoder = LabelEncoder()
            dataset[column] = encoder.fit_transform(dataset[column])
        
        # Replace the $ sign with empty space for values in each column
        for col in dataset.select_dtypes(include=['object']):  # Only process string columns
            # Replace dollar signs with spaces
            dataset[col] = dataset[col].str.replace('$', ' ', regex=False)
            cleaned_text = dataset[col].strip('$')  # Remove leading and trailing dollar signs
            cleaned_text = cleaned_text.replace('$', '\\$')  # Escape dollar signs within the text
            dataset[col] = cleaned_text    

    return datasets

numerically_datasets = convert_categorical_columns(datasets)

In [ ]:
from sklearn.preprocessing import MinMaxScaler # Scaling features to a range

def scale_datasets(datasets:list=numerically_datasets):
    """
        This function will scale the datasets.
        Parameters:
            datasets (list): The list of datasets
        Returns:
            datasets (list): The list of scaled datasets
    """
    for dataset in datasets:
        scaler = MinMaxScaler(feature_range=(0,10)) #StandardScaler()
        dataset = pd.DataFrame(scaler.fit_transform(dataset), columns=dataset.columns)
    return datasets

scaled_datasets = scale_datasets(numerically_datasets)

plot_data_distribution(scaled_datasets)

---
### Explore models and pick the best one(s)
For each dataset, go through each column and calculate its outliers. Visualize the outliers.


In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer # Imputation transformer for completing missing values


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest

In [ ]:

def plot_dataset_outliers(dataset: pd.DataFrame, dataset_outliers: list):
    """
        This function will plot the dataset and the outliers and display it:
        Parameters:
            dataset (pandas dataframe): The dataset
            dataset_outliers (list): The list of outliers
    """
    
    num_cols = 4
    num_rows = (len(dataset.columns) // num_cols) + (len(dataset.columns) % num_cols > 0)

    fig, axs = plt.subplots(num_rows, min(4, num_cols), figsize=(15, 5 * num_rows))
    axs = axs.flatten() if num_rows * num_cols > 1 else [axs]
    
    for idx, outliers in enumerate(dataset_outliers):
        col = dataset.columns[idx]
        # print(f"Dataset : {dataset.shape} {col}-Outliers: {len(outliers)}")

        axs[idx].plot(dataset[col], alpha=.9, linewidth=.8) # Plot the dataset values
        axs[idx].scatter(outliers, dataset[col][outliers], label='Outliers', alpha=.9, color='red', s=10) # Plot the outliers

        axs[idx].set_title(f'Dataset-{idx} {dataset.shape}', fontsize=8)
        axs[idx].set_xlabel(f'{col.replace("$", "")}', fontsize=8)
        axs[idx].set_ylabel('Value')
    
    plt.subplots_adjust(hspace=0.5) # Add a space between the plots
    plt.tight_layout()
    plt.show()


def detect_outliers(datasets: list):
    """
        This function will detect the outliers in the datasets.
        
        Parameters:
            datasets (list): The list of datasets
        
        Returns:
            list: The list of outlier scores
    """
    outlier_scores = []
    imputer = SimpleImputer(strategy='mean')  # Impute missing values with the mean
    model = IsolationForest(contamination=0.1) # 10% of the data are outliers

    for dataset in datasets: # For each dataset in the list of datasets
        dataset_outliers = [] # List to store the outliers for each dataset ... necessary?

        for col in dataset.columns: # For each column in the current dataset
            anomaly_indices = []
            
            imputer.fit(dataset[[col]]) # Train the imputer on the current column
            imputed_values = imputer.transform(dataset[[col]]) # Fill the missing values
            if imputed_values.shape[1] == 0: continue
            
            dataset[col] = pd.DataFrame(imputed_values, columns=[col]) # Replace the column

            model.fit(dataset[[col]]) # Fit the model to get baseline
            predictions = model.predict(dataset[[col]]) # The anomaly score of the input samples
            anomaly_indices = np.where(predictions == -1)[0] # Outliers are labeled -1 and inliers are labeled 1
            dataset_outliers.append(anomaly_indices) # Append the indices of the outliers

            # print(f'Dataset values and outliers: {dataset.shape} {len(anomaly_indices)} ..{.1*dataset.shape[0]}')
        
        # Plot anomalies for each column
        plot_dataset_outliers(dataset, dataset_outliers)
        # break # <- Uncomment to plot all datasets

    outlier_scores.append(dataset_outliers)
    
    return outlier_scores

outlier_scores = detect_outliers(scaled_datasets)
# outlier_scores

---
### Present solution

Build an app which takes as input _Link_ to a dataset or multiple datasets, and returns a plot of the columns and anomalies in each column

In [ ]:
%pip install protobuf typing_extensions cachetools
%pip install altair<6,>=4.0 click<9,>=7.0 gitpython!=3.1.19,<4,>=3.0.7 
%pip install pyarrow>=7.0 pydeck<1,>=0.8.0b4 rich<14,>=10.14.0 tenacity<10,>=8.1.0 toml<2,>=0.10.1
%pip install pillow==10.0.0
%pip install streamlit==1.18.1
%pip install streamlit_jupyter

%pip freeze > requirements.txt


import streamlit as st

In [ ]:
warnings.filterwarnings('ignore')

from streamlit_jupyter import StreamlitPandas as stp

In [ ]:
def load_dataset(urls:list):
    """
    Load a dataset .

    Args:
        url (str): The URL of the dataset.

    Returns:
        pd.DataFrame: A pandas DataFrame containing the dataset, or None if loading fails.
    """
    try:
        datasets = get_datasets(urls)
        datasets = remove_duplicates(datasets)
        datasets = remove_small_datasets(datasets)
        datasets = handle_missing_values(datasets)
        non_alphanumeric_chars = find_non_alphanumeric_data(datasets)
        datasets = replace_non_alphanumeric_data(datasets, non_alphanumeric_chars)
        datasets = clean_columns(datasets)

        numerically_datasets = convert_categorical_columns(datasets)
        scaled_datasets = scale_datasets(numerically_datasets)
        return scaled_datasets
    except Exception as e:
        st.error(f"Failed to load data from {urls}: {e}")
        return None

def detect_anomalies(df):
    """
    Detect anomalies in each feature of a DataFrame using Z-score.

    Args:
        df (pd.DataFrame): The DataFrame to analyze.

    Returns:
        dict: A dictionary with feature names as keys and lists of anomaly indices as values.
    """
    outlier_scores = detect_outliers(scaled_datasets)

    return outlier_scores


In [ ]:
st.title("Anomaly Detection in Datasets")

# Input: List of dataset links
dataset_links = st.text_area("Enter dataset URLs (one per line):")
if dataset_links:
    urls = dataset_links.strip().split(',')
    st.subheader(f"Datasets from input...")
        
    # Load the dataset
    df = load_dataset(urls)
    if df is not None:
        st.write("Data Preview:", df.head())
        
        # Detect anomalies
        anomalies = detect_anomalies(df)
        st.write('Outliers detected' if len(anomalies) > 0 else 'No outliers detected')
stp.render()

In [ ]:
def detect_anomalies_isolation_forest(df):
    """
    Detect anomalies using Isolation Forest.

    Args:
        df (pd.DataFrame): The DataFrame to analyze.

    Returns:
        dict: A dictionary with feature names as keys and lists of anomaly indices as values.
    """
    anomalies = {}
    for column in df.select_dtypes(include=[np.number]).columns:
        model = IsolationForest(contamination=0.05)  # Adjust contamination as needed
        df['anomaly'] = model.fit_predict(df[[column]])
        anomaly_indices = df[df['anomaly'] == -1].index.tolist()  # -1 indicates anomalies
        anomalies[column] = anomaly_indices
        df.drop('anomaly', axis=1, inplace=True)  # Clean up the DataFrame

    return anomalies

def main():
    st.title("Anomaly Detection in Datasets with Isolation Forest")

    dataset_links = st.text_area("Enter dataset URLs (one per line):")
    if dataset_links:
        urls = dataset_links.strip().split('\n')
        for url in urls:
            st.subheader(f"Dataset from: {url}")
            df = load_dataset(url)
            if df is not None:
                st.write("Data Preview:", df.head())
                anomalies = detect_anomalies_isolation_forest(df)
                st.write("Anomalies Detected:")
                for feature, indices in anomalies.items():
                    st.write(f"Feature '{feature}': {len(indices)} anomalies at indices {indices}")

if __name__ == "__main__":
    main()